In [1]:
#import library
import os
import tensorflow as tf
import tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Evaluator, Pusher
from tfx.proto import trainer_pb2
from tfx.dsl.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor

2025-03-02 01:59:22.495663: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Path untuk dataset dan output pipeline
USERNAME = "bernadettasri"
PIPELINE_ROOT = f"{USERNAME}-pipeline"
DATA_ROOT = os.path.join(os.getcwd(), "data")
MODEL_DIR = os.path.join(PIPELINE_ROOT, 'model')
# Buat direktori output jika belum ada
os.makedirs(PIPELINE_ROOT, exist_ok=True)

In [3]:
# Inisialisasi TFX Interactive Context
context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

In [4]:
# 1. ExampleGen: Mengimpor data CSV ke dalam pipeline
import shutil
csv_path = os.path.join(DATA_ROOT, "sentimentdataset.csv")
target_folder = os.path.join(DATA_ROOT, "sentimentdataset")
# Buat folder jika belum ada
os.makedirs(target_folder, exist_ok=True)
# Pindahkan file CSV ke dalam folder
shutil.move(csv_path, os.path.join(target_folder, "data.csv"))
# Jalankan ulang CsvExampleGen
example_gen = CsvExampleGen(input_base=target_folder)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [5]:
# 2. StatisticsGen: Menghasilkan statistik data
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [6]:
# 3. SchemaGen: Membuat skema berdasarkan statistik data
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'])
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [7]:
# 4. ExampleValidator: Memvalidasi data terhadap skema
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [ ]:
from tensorflow_metadata.proto.v0 import schema_pb2
import tensorflow as tf
from google.protobuf import text_format

schema_pbtxt_path = "//Users/bernadetta/Desktop/MLOps1/bernadettasri-pipeline/SchemaGen/schema/3/schema.pbtxt"
schema_pb_path = "/Users/bernadetta/Desktop/MLOps1/bernadettasri-pipeline/SchemaGen/schema/3/schema.pb"

# Load schema dari pbtxt
schema = schema_pb2.Schema()
with tf.io.gfile.GFile(schema_pbtxt_path, "r") as f:
    schema_text = f.read()
    text_format.Merge(schema_text, schema)

# Simpan sebagai .pb
with tf.io.gfile.GFile(schema_pb_path, "wb") as f:
    f.write(schema.SerializeToString())

print(f"Schema telah dikonversi ke {schema_pb_path}")

In [ ]:
import os

schema_pbtxt_path = "/Users/bernadetta/Desktop/MLOps1/bernadettasri-pipeline/SchemaGen/schema/3/schema.pbtxt"  # Ubah dengan path yang benar

if os.path.exists(schema_pbtxt_path):
    os.remove(schema_pbtxt_path)
    print(f"{schema_pbtxt_path} berhasil dihapus.")
else:
    print(f"{schema_pbtxt_path} tidak ditemukan.")

In [ ]:
from tensorflow_metadata.proto.v0 import schema_pb2
import tensorflow as tf

schema_pb_path = "/Users/bernadetta/Desktop/MLOps1/bernadettasri-pipeline/SchemaGen/schema/3/schema.pb"

# Load schema dari .pb dengan mode biner
schema = schema_pb2.Schema()
with tf.io.gfile.GFile(schema_pb_path, "rb") as f:
    schema.ParseFromString(f.read())

print("Schema loaded successfully.")

In [8]:
# 5. Transform: Melakukan preprocessing data
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath("preprocessing.py")  
)
context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying preprocessing.py -> build/lib
installing to /var/folders/3f/hhpf71r53758_j1y7jxv9h840000gp/T/tmp5y7er0f7
running install
running install_lib
copying build/lib/preprocessing.py -> /var/folders/3f/hhpf71r53758_j1y7jxv9h840000gp/T/tmp5y7er0f7
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /var/folders/3f/hhpf71r53758_j1y7jxv9h840000gp/T/tmp5y7er0f7/tfx_user_code_Transform-0.0+e08a4c2392b9dcc4fed1f6a6b287f6dc8b9d0

/usr/local/Caskroom/miniconda/base/envs/mlops1/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


Processing ./bernadettasri-pipeline/_wheels/tfx_user_code_Transform-0.0+e08a4c2392b9dcc4fed1f6a6b287f6dc8b9d01e629aeeba6c8f6c54f9fa57618-py3-none-any.whl
Processing ./bernadettasri-pipeline/_wheels/tfx_user_code_Transform-0.0+e08a4c2392b9dcc4fed1f6a6b287f6dc8b9d01e629aeeba6c8f6c54f9fa57618-py3-none-any.whl
Processing ./bernadettasri-pipeline/_wheels/tfx_user_code_Transform-0.0+e08a4c2392b9dcc4fed1f6a6b287f6dc8b9d01e629aeeba6c8f6c54f9fa57618-py3-none-any.whl
Instructions for updating:
Use ref() instead.


2025-03-02 02:00:33.730671: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: bernadettasri-pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/f73a2641a12245e38effa0ad48a87020/assets


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [9]:
# 6. Trainer: Melatih model menggunakan TensorFlow
from tfx.dsl.components.common.importer import Importer
from tfx.types import standard_artifacts
TRAINER_MODULE_FILE = os.path.join(PIPELINE_ROOT, 'trainer.py')
with open(TRAINER_MODULE_FILE, 'w') as f:
    f.write("""
import tensorflow as tf
import tensorflow_transform as tft
from google.protobuf import text_format
from tensorflow_metadata.proto.v0 import schema_pb2
from tensorflow_transform.tf_metadata import schema_utils
import os

# Model function
def _build_keras_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# Function untuk parsing TFRecord berdasarkan schema
def _parse_function(serialized_example, schema):
    # Konversi schema menjadi feature_spec
    feature_spec = schema_utils.schema_as_feature_spec(schema).feature_spec
    parsed_example = tf.io.parse_single_example(serialized_example, feature_spec)
    return parsed_example

def run_fn(fn_args):
    schema_path = "/Users/bernadetta/Desktop/ProyekMLOps1/bernadettasri-pipeline/SchemaGen/schema/6/schema.pb"
    schema = schema_pb2.Schema()

    # Pastikan path schema benar
    if not tf.io.gfile.exists(schema_path):
        raise FileNotFoundError(f"Schema file not found: {schema_path}")

    # Membaca schema
    with tf.io.gfile.GFile(schema_path, "rb") as f:
        schema.ParseFromString(f.read())
    print("Schema parsed successfully.")

    # Ambil path dataset dari Transform
    train_dataset_path = fn_args.train_files[0]  
    eval_dataset_path = fn_args.eval_files[0]

    print("Train dataset path:", train_dataset_path)
    print("Eval dataset path:", eval_dataset_path)

    if not tf.io.gfile.exists(train_dataset_path) or not tf.io.gfile.exists(eval_dataset_path):
        raise FileNotFoundError("Train or Eval dataset files not found!")

    # Parsing dataset menggunakan schema
    train_dataset = (tf.data.TFRecordDataset(train_dataset_path, compression_type="GZIP")
                     .map(lambda x: _parse_function(x, schema))
                     .batch(32)
                     .shuffle(1000))
    
    eval_dataset = (tf.data.TFRecordDataset(eval_dataset_path, compression_type="GZIP")
                    .map(lambda x: _parse_function(x, schema))
                    .batch(32))

    # Bangun model
    model = _build_keras_model()

    # Training
    model.fit(train_dataset, validation_data=eval_dataset, epochs=5)

    # Simpan model
    model.save(fn_args.serving_model_dir, save_format='tf')

""")

In [14]:
from tensorflow_metadata.proto.v0 import schema_pb2
import tensorflow as tf
import os

# Path ke schema.pbtxt
schema_uri = "bernadettasri-pipeline/SchemaGen/schema/3"
schema_pbtxt_path = os.path.join(schema_uri, "schema.pbtxt")
schema_pb_path = os.path.join(schema_uri, "schema.pb")

# Load schema dari pbtxt
schema = schema_pb2.Schema()
with tf.io.gfile.GFile(schema_pbtxt_path, "r") as f:
    schema_text = f.read()
    tf.compat.v1.logging.info("Loaded schema.pbtxt")
    from google.protobuf import text_format
    text_format.Merge(schema_text, schema)  # Pakai text_format yang benar

# Simpan sebagai .pb
with tf.io.gfile.GFile(schema_pb_path, "wb") as f:
    f.write(schema.SerializeToString())

print(f"Schema telah dikonversi ke {schema_pb_path}")


INFO:tensorflow:Loaded schema.pbtxt


INFO:tensorflow:Loaded schema.pbtxt


Schema telah dikonversi ke bernadettasri-pipeline/SchemaGen/schema/3/schema.pb
